In [1]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import timedelta
from matplotlib import pyplot as plt
from statsmodels.tsa.vector_ar.var_model import VAR
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.arima_model import ARIMA
from math import sqrt
from math import log
from math import isnan
from math import isinf
from tabulate import tabulate
import time
import pickle
import warnings
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree.export import export_text
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score
from statistics import stdev as std
from IPython.display import display, clear_output
import random
from sklearn.model_selection import GridSearchCV

In [2]:
# simple timer to time events
class Timer:
    def __init__(self):
        self.start = time.time()
        
    def get(self):
        value = time.time() - self.start
        self.start = time.time()
        return value

In [3]:
# read and mungle data with number of cases from covid-19 file from european cdc:
# https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide
# keeps the columns ['Date', 'Cases', 'Country']
# changes name of some countries to match population data from the UN:
# https://population.un.org/wpp/Download/Standard/CS
# drops some regions that do not exist in the UN population file
# accepts data from a country if all data have a number of total days greater than days_total
# accepts data from a country if all data have a maximum number of cases greater than cases_max
# returns data sorted by ascending date while keeping grouped by country
# rows which did not have an entry for that date have the number of cases set to -1
def readCOVID19(path, days_total=2, cases_max=1):
    
    covid19_cases = pd.read_csv('COVID-19_cases_20200320.csv')

    # remove all attributes except country and number of cases by date
    covid19_cases = covid19_cases[['DateRep', 'Cases', 'Countries and territories']]
    covid19_cases.columns = ['Date', 'Cases', 'Country']
    covid19_cases['Date'] = pd.to_datetime(covid19_cases.Date)

    # organize by day number
    data = covid19_cases.sort_values(by=['Country','Date'])

    # fix something so the cases an population .csv file names match up by country name
    data['Country'] = data['Country'].str.replace('_', ' ')
    data['Country'] = data['Country'].str.replace('CANADA', 'Canada') # Canada was entered in all-caps?
    data = data.loc[data['Country'] != 'Cote dIvoire']
    data = data.loc[data['Country'] != 'Kosovo']
    data = data.loc[data['Country'] != 'Cases on an international conveyance Japan']
    
    # mungle loop that chooses to accept data based on previous parameters
    # also redoes the 'Date' column to be 'Day' column which is number of days since first outbreak in that country
    reranked = [] # array of new rows to make new dataframe at end
    tosort = {} # dictionary that will sort country data in descending order of max number of cases
    lastCountry = '' # keep track of country in previous row 
    lastDate = '' # keep track of date in previous row
    dayNum = 1 # start with day 1
    firstCase = False # log if this is the first case for given country
    temp = [] # temp array to hold data points from a country, will either be accepted or rejected
    skip = False # control if temp array is accepted or rejected
    maxCases = 0 # keep track of maximum number of cases for country, to sort countries by severity
    maxes = [] # keep track of all max cases for all countries
    for index, row in data.iterrows():
        
        # get column values for this row
        thisDate = row[0]
        cases = row[1]
        country = row[2]

        # check if this is a new country from previous row
        if country not in lastCountry:
            # choose to either accept or reject data from last country
            if len(temp) >= days_total and maxCases > cases_max:
                # remove last entry (uncertainty if day was complete)
                temp.pop(-1)
                tosort[lastCountry] = [temp, maxCases] # save temp array to the tosort dictionary if accepted
                maxes.append(maxCases)  # save list of max cases to array if accepted
            # make a new temp array for this new country's data, and reinitialize other vars
            temp = []
            skip = False
            firstCase = False
            dayNum = 1
            maxCases = 0

        # check if this date is first case for this country
        if cases > 0 and not firstCase:
            lastDate = thisDate
            firstCase = True

        # if we are on or passed our first date, then log data
        if firstCase:
            # get change in days betwen this date and last date
            deltaDays = (thisDate - lastDate).days
            if deltaDays < 0:
                print('Error data is not properly sorted by date', country, thisDate)
            # check if the days inbetween this row and last row exceeds our threshold to accept
            else:
                # keep track of max vases
                maxCases = max(cases, maxCases)
                # add for each day in between
                for i in range(deltaDays-1):
                    # add data to temp array
                    temp.append([dayNum + i + 1, -1, country, lastDate + timedelta(days=1+i), False])
                # add for this day
                dayNum += deltaDays
                temp.append([dayNum, cases, country, thisDate, True])
                
        # log previous date and country
        lastDate = thisDate
        lastCountry = country
        
    # add values from temp array for last country if we are to accept
    if len(temp) >= days_total and maxCases > cases_max:
        # remove last entry (uncertainty if day was complete)
        temp.pop(-1)
        tosort[lastCountry] = [temp, maxCases]
        maxes.append(maxCases)

    # sort countries now by highest number of cases (mostly for visual purposes of plots)
    while len(maxes) > 0:
        # get next max value
        thisMax = max(maxes)
        # look for countries with max value
        pops = [] # keep track of what country to remove from tosort
        for country in tosort:
            # get values for thsi country
            rows = tosort[country][0]
            maxCases = tosort[country][1]
            # if this has max value then add to final rows
            if thisMax == maxCases:
                reranked = reranked + rows
                pops.append(country) 
        # remove countries that were aded from temporary tosort dict
        for country in pops:        
            tosort.pop(country)
        # remove this max from list of max cases
        maxes = [x for x in maxes if x < thisMax]

    # make new data frame with ranked, mungled data
    data = pd.DataFrame(reranked, columns=['Day', 'Cases', 'Country', 'Date', 'Valid'])
    
    return data

In [4]:
# plots all COVID19 data 5 at a time (5 countries per plot)
# @data is pandas data frame with 3 columns ['Day', 'Cases', 'Country'] 
    # grouped by country, reformated by day# since first outbreak, sorted by ascending day
# @xlabel is x-axis label to output on plots
# @ylabel is y-axis label to output on plots
def plot5(data, xlabel='Day', ylabel='Cases'):
    # redo plot every 5 countries
    lastCut = 0 # get rows to cut for a plot of 5 countries
    lastCountry = data.at[0,'Country'] # keep track of what the last country in the last row was
    while lastCut < len(data) - 1:
        
        # init cut in dataframe to last cut from from plot
        thisCut = lastCut
        
        # iterate through rows and add to this cut until we have reached a 6th
        nCountries = 0 # keep track of how many countries are in current cut
        for i in range(lastCut, len(data), 1):
            # get country this row is frome
            thisCountry = str(data.at[i,'Country'])
            # check if we have added all rows from a new country
            if thisCountry not in lastCountry or i == len(data) - 1:
                nCountries += 1
            # log what country the previous row is from
            lastCountry = thisCountry
            # check if we have 5 countries worth of data in cut
            if nCountries > 4:
                break
            # otherwise increase cut by another row
            thisCut += 1
        
        # plot this cut
        fig, ax = plt.subplots(figsize=(15,7))
        data_temp = data.iloc[lastCut:thisCut]
        data_temp.loc[data_temp['Cases'] > -1].groupby(['Day', 'Country']).sum()['Cases'].unstack().plot(ax=ax)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        
        # log previous cut
        lastCut = thisCut

In [5]:
# returns Root Mean Squared Error (RMSE) between two lists
def root_mean_squared_error(test, predictions):
    if len(test) == len(predictions):
        error = 0.
        for i in range(len(test)):
            error += (test[i] - predictions[i]) ** 2
        error /= float(len(test))
        return sqrt(error)

In [6]:
# returns Root Mean Squared Percent Error (RMSPE) between two lists
def root_mean_squared_percent_error(test, predictions):
    if len(test) == len(predictions):
        error = 0.
        for i in range(len(test)):
            error += ((test[i] - predictions[i]) / test[i]) ** 2
        error /= float(len(test))
        return 100 * sqrt(error)

In [7]:
# split into dictionary and remove 0s
# data is pandas data frame following structure returned from readCOVID19()
# returns a dictionary that maps country to a list of cases ordered by day number
def data_prep_ARIMA(data):
    # loop through data and create lists by country name
    data_dict = {} # init dictionary
    for i in range(len(data)):
        # get country from row 
        country = data.at[i,'Country']
        # check if country in dictionary
        if country not in data_dict:
            # if not, then intit a new list
            data_dict[country] = []
        # check if cases number from row is positive non-zero
        if data.at[i,'Cases'] > 0:
            # add to list for this country if positive non-zero
            data_dict[country].append(data.at[i,'Cases'])
        # otherwise add nan for ARIMA to handle missing case
        else:
            data_dict[country].append(np.nan)
    return data_dict

In [8]:
# plot autocorrelation for number of lags in time-series
# one plot for each country
# @data_dict is a dictionary mapping countries to list of cases
def autocorrelation_plots(data_dict):
    for country in data_dict:
        autocorrelation_plot([x for x in data_dict[country] if not isnan(x)])
        plt.title(country)
        plt.show()

In [9]:
# runs an ARIMA test on data over range of p, d, q
# @data_dict is a dictionary mapping countries to list of cases
# @nDays_range is range of days to check in viewing window
# @derivative_range is range of derivatives to check (0 or 1 make most sense)
# @lag_vals a list of two numbers to add to nDays during iteration:
    # first index is starting value for range
    # second index is ending value for range (non-inclusive)
# @output_ARIMA is path to file to output results as pickle 
# @header is a list of strings for header to table that is output
def iter_ARIMA(data_dict, nDays_range, derivative_range, lag_range, output_ARIMA):
    # initialize some vars
    header = ['Lags', 'Derv', 'Days', 'RMSE %', 'RMSE', 'Accuracy %', '% Converged', 'Exec Time (s)']
    warnings.filterwarnings('ignore') # ignore warnings that show when datapoints do not converge
    rmspe = {} # log rmspe for each iteration, by country
    rmse = {} # log rmse for each iteration, by country
    accuracy = {} # log if accurately went up or down
    params = {} # log parameters, by country
    allTests = {} # log test case values, by country
    allPredictions = {} # log predicted case values, by country
    # init lists that keeps track of all results 
    rmspe['all'] = [] 
    rmse['all'] = []
    accuracy['all'] = []
    params['all'] = []
    allTests['all'] = []
    allPredictions['all'] = []
    table = [] # table for output, previewing live results
    times = [] # list of how long each iteration takes
    timer = Timer() # init timer
    
    # iterate through desired number of days in viewing window
    for nDays in nDays_range:
        # iterate through degree of derivative to take
        for derivative in derivative_range:
            # iterate through number of lags to look at (number of windows to predict next value)
            for lag in lag_range:
                theseTests = [] # temp list logging tests
                thesePredictions = [] # temp list logging predictions
                nConverged = 0 # log number of datapoints which converged
                nTried = 0 # log number of datapoints attempted to converge
                nCorrect = 0 # log number of correctly predicted points to go up or down
                
                # run ARIMA on each country
                for country in data_dict:
                    # get entire time-series of datapoints
                    X = data_dict[country]
                    # check if time-series is long enough
                    if len(X) <= lag:
                        continue
                    # get first index for chunking into train and test values
                    size = 1 + lag + derivative
                    # chunk into train and test
                    train, test = X[0:size], X[size:len(X)]
                    test_conv = [] # keep track of test values which converge
                    history = [x for x in train] # accumalating history of all values in train chunk
                    predictions = [] # keep track of predicted values for those which converge
                    nSuccess = len(test) # list the number of success (subtract when error happens from non-convergence)
                    this_nCorrect = 0
                    # test on all datapoints in test chunk
                    for t in range(len(test)):
                        nTried += 1 # inc data point tried
                        # try to fit ARIMA, error will throw if not converged
                        try:
                            # get current history
                            hist_temp = history
                            # add to future history
                            history.append(test[t])
                            # make ARIMA model
                            model = ARIMA(hist_temp, order=(lag, derivative, nDays))
                            # fit ARIMA model
                            model_fit = model.fit(maxiter=1000)
                            # predict next value in test chunk
                            output = model_fit.forecast()
                            # if we are here than converged, add to success lists
                            test_conv.append(test[t])
                            predictions.append(output[0])
                            # check if we accurately went up or down 
                            if test[t] > hist_temp[-1] and output[0] > hist_temp[-1]:
                                this_nCorrect += 1
                            elif test[t] <= hist_temp[-1] and output[0] <= hist_temp[-1]:
                                this_nCorrect += 1
                        # if an error is thrown dec the number of successes
                        except Exception as e:
                            nSuccess -= 1
                    # if atleast one data point converged then log results for this country
                    if nSuccess > 0:
                        nConverged += nSuccess # inc number converged
                        nCorrect += this_nCorrect # inc number correct
                        theseTests = theseTests + test_conv # add to test list
                        thesePredictions = thesePredictions + predictions # add to prediction list
                        error1 = root_mean_squared_error(test_conv, predictions) # calc rmse
                        error2 = root_mean_squared_percent_error(test_conv, predictions) # calc rmspe
                        error3 = 100. * this_nCorrect / nSuccess # calc accuracy
                        # check if we need to init a list for this country in results dict
                        if country not in rmse:
                            rmse[country] = []
                            rmspe[country] = []
                            accuracy[country] = []
                            params[country] = []
                            allTests[country] = []
                            allPredictions[country] = []
                        # add results to respective dict for this country
                        rmse[country].append(error1)
                        rmspe[country].append(error2)
                        accuracy[country].append(error3)
                        params[country].append([lag,derivative,nDays])
                        allTests[country].append(test_conv)
                        allPredictions[country].append(predictions)

                # log results for all countries
                this_time = timer.get() # check timer
                times.append(this_time) # add time for iteration
                # check if any datapoints converged for this iteration then add to results
                if nConverged > 0:
                    error1 = root_mean_squared_error(theseTests, thesePredictions) # calc rmse for all countries
                    error2 = root_mean_squared_percent_error(theseTests, thesePredictions) # same for rmspe
                    error3 = 100. *  nCorrect / nConverged
                    params['all'].append([lag, derivative, nDays]) # add parameters checked this iter
                    allTests['all'].append(theseTests) # add all test datapoints to list 
                    allPredictions['all'].append(thesePredictions) # add all predictions to list
                    rmse['all'].append(error1) # add rmse
                    rmspe['all'].append(error2) # add rmspe
                    accuracy['all'].append(error3) # add rmspe
                    table.append([ lag, derivative, nDays, round(error2, 2), round(error1, 2), round(error3, 2)
                                  , round(100 * nConverged / nTried, 2), this_time ]) # add new row with results to table
                    clear_output() # clear output on screen for new table
                    print(tabulate(table, headers=header)) # print new table
    
    # write pickle file for easy reading later
    ARIMA_results = [rmspe, rmse, accuracy, params, allTests, allPredictions, table]
    with open(output_ARIMA, 'wb') as outfile:
        pickle.dump(ARIMA_results, outfile)
        
    return ARIMA_results

In [10]:
# reads and displays arima output from previous runs
# @input_ARIMA list of paths to files to read pervious results from
# @header list of strings that are header to table
def read_ARIMA(input_ARIMA):
    header = ['Lags', 'Derv', 'Days', 'RMSE %', 'RMSE', 'Accuracy %', '% Converged', 'Exec Time (s)']
    ARIMA_results = {}
    for arima in input_ARIMA:
        with open(arima, 'rb' ) as infile:
            ARIMA_results[arima] = pickle.load(infile)
            print(tabulate(ARIMA_results[arima][-1], headers=header))
    return ARIMA_results

In [11]:
# takes the moving sum of data
# @data is pandas data frame split by ('Day', 'Cases', 'Country') 
    # assumed to be grouped by country name and sorted by day number in ascending order
# @days_moving is window size of moving sum
# returns dataFrame similar to @data except days are moving sums of those days 
    # there will be days_moving-1 less rows for each country
def moving_sum(data, days_moving):
    # take moving sum
    newRows = [] # keep track of new rows to make new data frame at end
    lastCountry = '' # keep track of country of last row, tells when to reset moving window
    counter = 0 # count number of rows viewed from country
    idx = 0 # track index of moving window
    window = [None] * days_moving # init moving window
    for i in range(len(data)):
        # get country of this row
        country = str(data.at[i, 'Country'])
        # check if this is the next country
        if country not in lastCountry:
            # if so then reset counter
            counter = 0
        # set value at current window location
        window[idx] = max(0, data.at[i, 'Cases'])
        # increment revovling window
        idx += 1
        if idx >= days_moving:
            idx = 0
        # inc number of datapoints in this country
        counter += 1
        # if enough datapoints, then add to newRows array to later build dataframe
        if counter >= days_moving:
            newRows.append([data.at[i, 'Day'], sum(window), country, data.at[i, 'Date'], data.at[i, 'Cases'] >= 0])
        # log country of last row
        lastCountry = country
        
    # create new dataframe from moving window rows
    data2 = pd.DataFrame(newRows, columns=['Day', 'Cases', 'Country', 'Date', 'Valid'])
    return data2

In [12]:
# creates X and Y datapoints, for train and test sets
# @data is a dataframe following the architecture from read_COVID19()
# @window_size is size of moving sum
# @lags is number of previous datapoints to put into X
# @outputs is number of following datapoints after X to put into Y
# @sample_name name of sample used for output
# @label_type determines how to set Y, it is either:
    # 'next' for count after X (DEFAULT)
    # 'change' for the difference of count after X
    # 'class' for either binary 0 for a decrease after X or 1 for increase
# @test_split is optional value between [0,1] to decide the percent of test split
# @deriv will either take or not take 1st derivative
# return data, Xs, Ys, countries, days all mapped by either 'train' or 'test' in one dict
def prepare_MLA(data, window_size, lags, outputs, sample_name, label_type='next', test_split=0, deriv=False):
    
    # set params list
    params = {'window_size' : window_size, 'lags' : lags, 'outputs' : outputs
              , 'label_type' : label_type}
        
    # calculate moving sum of data
    if window_size > 1:
        data = moving_sum(data, window_size)
    
    # create empty matricies to be filled with data chunks
    Xs = {'train' : [], 'test' : []} # feature vectors
    Ys = {'train' : [], 'test' : []} # labels
    countries = {'train' : [], 'test' : []} # country names
    days = {'train' : [], 'test' : []} # day numbers of label

    # move through all rows in data
    for i in range(len(data)):
        
        # get country for this row
        country = str(data.at[i, 'Country'])

        # check for out of bounds 
        if i + lags + outputs > len(data):
            break
        elif deriv and i == 0:
            continue
        elif deriv and data.at[i-1, 'Country'] != country:
            continue
        
        # check if enough consequetive values are from same country
        if data.at[i + lags + outputs - 1, 'Country'] != country:
            continue
        
        # check if all data points are valid
        skip = False
        
        # add lags to feature vector to make a new chunk
        x = []
        day = [] # get day(s) of data and label
        for j in range(i, i + lags, 1):
            if not data.at[j, 'Valid']:
                skip = True
                break
            if deriv:
                x.append(data.at[j, 'Cases'] - data.at[j-1, 'Cases'])
            else:
                x.append(data.at[j, 'Cases'])
            day.append(data.at[j, 'Day'])
        if skip:
            continue

        # get labels for number of outputs
        y = []
        for j in range(i + lags, i + lags + outputs, 1):
            if not data.at[j, 'Valid']:
                skip = True
                break
            if label_type in 'next':
                y.append(int(data.at[j, 'Cases']))
            elif label_type in 'change':
                y.append(int(data.at[j, 'Cases']) - int(data.at[j - 1, 'Cases']))
            elif label_type in 'class':
                y.append(1 if int(data.at[j, 'Cases']) > int(data.at[j - 1, 'Cases']) else 0)
            day.append(int(data.at[j, 'Day']))
        if skip:
            continue
        
        # check if we add to train or test set
        addto = 'train'
        if random.random() < test_split:
            addto = 'test'
        
        # add chunk to matrices
        Xs[addto].append(x)
        Ys[addto].append(y)
        countries[addto].append(country)
        days[addto].append(day)
        
    #write pickle file
    pickle_out = {'data' : data, 'Xs' : Xs, 'Ys' : Ys, 'countries' : countries, 'days' : days, 'params' : params}
    with open('sample_' + sample_name, 'wb') as outfile:
        pickle.dump(pickle_out, outfile)
        
    return pickle_out

In [20]:
# standardizes data to have a mean of 0 and standard deviation of 1 for each day#
# @samples is a dict of map of chunks as returned from prepare_MLA()
# @means and @stds are the means and standard deviations mapped by day# for each sample
    # if these are passed in as parameters then the function will transform the data
    # if these are empty then the function will calculate them, transform data, and return means and stds
    # means and stds are calculated from train data, then used to transform train and test data
# @std_labels is a bool that controls to standardize labels or not
def standardize(samples, means = {}, stds = {}, std_labels=False):
   
    new_samples = {}
    for sample_name in samples:
        sample = samples[sample_name]
        
        # create new empty matricies to be filled with standardized data chunks
        new_Xs = {'train' : [], 'test' : []} # feature vectors
        new_Ys = {'train' : [], 'test' : []} # labels

        # get old data matrices
        X = sample['Xs']['train']
        Y = sample['Ys']['train']
        countries = sample['countries']['train']
        days = sample['days']['train']
        params = sample['params']

        # map if we have used this country/day already in sum
        used = {}

        # find means and std and transform data
        if sample_name not in means:
            means[sample_name] = {}
            stds[sample_name] = {}
            # get values for each day from train data
            byDay = {}
            for idx, x in enumerate(X):
                # get data for this chunk
                x = X[idx]
                y = Y[idx]
                day = days[idx]
                #print(x, y, day)
                country = countries[idx]
                # log values to calc averages/stds from feature vectors
                for j, x_j in enumerate(x):
                    d = day[j]
                    if country not in used:
                        used[country] = {}
                    if d not in used[country]:
                        used[country][d] = True
                        if d not in byDay:
                            byDay[d] = []
                        byDay[d].append(x_j)
                if std_labels:
                    # log values to calc averages/stds from labels
                    for j, y_j in enumerate(y):
                        d = day[len(x) + j]
                        if country not in used:
                            used[country] = {}
                        if d not in used[country]:
                            used[country][d] = True
                            if d not in byDay:
                                byDay[d] = []
                            byDay[d].append(y_j)

            # now transform train data by calc means for each vector/label
            for idx, x in enumerate(X):
                # get data for this chunk
                x = X[idx]
                y = Y[idx]
                day = days[idx]
                try:
                    # calc averages/stds of other days and transform feature vectors
                    new_x = []
                    for j, x_j in enumerate(x):
                        d = day[j]
                        sum_j = sum(byDay[d]) - x_j
                        len_j = len(byDay[d]) - 1
                        mean = sum_j / len_j
                        std = ( (sum([((v - mean) ** 2) for v in byDay[d]]) - (x_j - mean) ** 2) / len_j) ** 0.5
                        new_x.append((x_j - mean)/std)
                        if isnan(new_x[j]) or isinf(new_x[j]):
                            dummy = byDay[-2]
                    # calc averages/stds of other days and transform labels
                    new_y = []
                    if std_labels:
                        for j, y_j in enumerate(y):
                            d = day[len(x) + j]
                            sum_j = sum(byDay[d]) - y_j
                            len_j = len(byDay[d]) - 1
                            mean = sum_j / len_j
                            std = ( (sum([((v - mean) ** 2) for v in byDay[d]]) - (y_j - mean) ** 2) / len_j) ** 0.5
                            new_y.append((y_j - mean)/std)
                            if isnan(new_y[j]) or isinf(new_y[j]):
                                dummy = byDay[-2]
                    else:
                        new_y = y
                except Exception:
                    continue
                else:
                    new_Xs['train'].append(new_x)
                    new_Ys['train'].append(new_y)

                # calculate means and stds from training data
                for d in byDay:
                    means[sample_name][d] = sum(byDay[d]) / len(byDay[d])
                    stds[sample_name][d] = (sum([((v - means[sample_name][d]) ** 2) for v in byDay[d]]) / len(byDay[d])) ** 0.5

        # transform train data with passed in means and stds
        else: 
            for idx, x in enumerate(X):  
                # get data for this chunk
                x = X[idx]
                y = Y[idx]  
                day = days[idx]
                try:
                    # transform feature vectors
                    new_x = []
                    for j, x_j in enumerate(x):
                        d = day[j]
                        new_x.append((x_j - means[sample_name][d])/stds[sample_name][d])
                        if isnan(new_x[j]) or isinf(new_x[j]):
                            dummy = byDay[-2]
                    # transform labels
                    new_y = []
                    if std_labels:
                        for j, y_j in enumerate(y):
                            d = day[len(x) + j]
                            new_y.append((y_j - means[sample_name][d])/stds[sample_name][d])
                            if isnan(new_y[j]) or isinf(new_y[j]):
                                dummy = byDay[-2]
                    else:
                        new_y = y
                except Exception:
                    continue
                else:
                    new_Xs['train'].append(new_x)
                    new_Ys['train'].append(new_y)

        # transform test data
        for idx, x in enumerate(sample['Xs']['test']):
            # get data for this chunk
            x = sample['Xs']['test'][idx]
            y = sample['Ys']['test'][idx]
            day = sample['days']['test'][idx]
            try:
                # transform feature vectors
                new_x = []
                for j, x_j in enumerate(x):
                    d = day[j]
                    new_x.append((x_j - means[sample_name][d])/stds[sample_name][d])
                    if isnan(new_x[j]) or new_x(new_y[j]):
                        dummy = byDay[-2]
                # transform labels
                new_y = []
                if std_labels:
                    for j, y_j in enumerate(y):
                        d = day[len(x) + j]
                        new_y.append((y_j - means[sample_name][d])/stds[sample_name][d])
                        if isnan(new_y[j]) or isinf(new_y[j]):
                            dummy = byDay[-2]
                else:
                    new_y = y
            except Exception:
                continue
            else:
                new_Xs['test'].append(new_x)
                new_Ys['test'].append(new_y)

        #write pickle file
        new_samples[sample_name] = {'data' : data, 'Xs' : new_Xs, 'Ys' : new_Ys, 'countries' : countries
                                    , 'days' : days, 'params' : params, 'sample_name' : sample_name}
        with open('sample_std_' + sample_name, 'wb') as outfile:
            pickle.dump(new_samples[sample_name], outfile)

    return new_samples, means, stds

In [14]:
# runs cross-validation multiple times for all samples in output_samples
# reads cross-validation results already ran from inpu_samples
# @mlas is a list of string names of what mlas to use
# @disp is a bool to either show output or not
# function will return all output and input results and plot them
def mla_class_results(mlas, output_samples=[], input_sample_names=[], disp=True):
    all_results = {}
    
    # run new and plot results, if samples dictionary is filled
    warnings.filterwarnings('ignore') # ignore warnings that show when models do not converge
    for sample_name in output_samples:
        sample = output_samples[sample_name]
        results = {}
        results['sample_name'] = sample_name
        if disp:
            print('sample ' + sample_name + ' ...')
        for mla in mlas:
            if disp:
                print('MLA ' + mla + ' ...')
            results[mla] = crossValidation_class(mla, sample['Xs']['train'], sample['Ys']['train'], 100, 5
                                                 , (max(int((sample['params']['lags'])/2), 1)), disp )
        all_results[sample_name] = results
        if disp:
            plotResults_class(results, mlas) 

        #write pickle file with results
        with open('MLA_class_' + sample_name, 'wb') as outfile:
            pickle.dump(results, outfile)

    # read and plot results if inputs is not empty
    for sample_name in input_sample_names:
        with open('MLA_class_' + sample_name, 'rb' ) as infile:
            results = pickle.load(infile)
            all_results[sample_name] = results
            if disp:
                plotResults_class(results, mlas)
                
    return all_results

In [15]:
# reads pickle files of premade samples used MLA
# @input_sample is list of file paths to pickle files
# @returns list of samples
def read_samples(sample_names):
    samples = {}
    for sample_name in sample_names:
        with open('sample_' + sample_name, 'rb' ) as infile:
            samples[sample_name] = pickle.load(infile)
    return samples

In [16]:
# runs cross validation of classification MLAs
# @name is name of MLA (see if statements below)
# @X is matrix of training features
# @y are labels
# @nIters is the number of times to re-run cross-validation
# @nFolds is number of folds in cross-validation
# @layers is a list of layer sizes to use for MLP
# @disp will display results
# returns list with average accuracy for each iteration
def crossValidation_class(name, X, y, nIters, nFolds, layers, disp=True):
    
    if disp:
        dis = display(f'running MLA {name}',display_id=True)
    
    # create dummy Machine Learning Algorithm (MLA) object
    mla = tree.DecisionTreeClassifier()
    
    # keep track of results
    means = [None] * nIters
    #stds = [None] * nIters
    
    # get results
    for i in range(nIters):
         # create MLA with new random seed
        if 'DTr' in name:
            mla = DecisionTreeClassifier(random_state = i)
        elif 'NBa' in name:
            mla = GaussianNB()
        elif 'RFo' in name:
            mla = RandomForestClassifier(random_state = i, n_estimators=100)
        elif 'ETr' in name:
            mla = ExtraTreesClassifier(random_state = i, n_estimators=100)
        elif 'SVM' in name:
            mla = SVC(random_state = i, gamma='auto', probability=True)
        elif 'KNN' in name:
            mla = KNeighborsClassifier(n_neighbors=2)
        elif 'MLP' in name:
            mla = MLPClassifier(random_state = i, hidden_layer_sizes=layers, max_iter=10000)
            
        #print(X, y, 'EOL')
        scores = cross_val_score(mla, X, y, cv=nFolds)
        means[i] = scores.mean()
        #stds[i] = std(scores)
        if disp:
            dis.update(f'iteration {i+1} of {nIters} done')
    if disp:
        dis.update(f'Average classification accuracy = {100. * sum(means) / len(means):.2f}%')
        
    return means

In [17]:
# plots the results returned from crossValidation_class()
# @results are results returned from crossValidation_class()
# @mlas are list of mlas used in results to output
def plotResults_class(results, mlas):
    x_labels = []
    for mla in mlas:
        x_labels.append(mla)
        x_labels.append(' ')
        x_labels.append(' ')
        x_labels.append(' ')
    x_nums = [x for x in range(5*len(mlas))]
    fig = plt.figure(figsize=(7.5,5))
    plt.xticks(x_nums, x_labels, rotation='vertical', fontsize=20)
    plt.yticks([x for x in range(75, 101, 5)], fontsize=20)
    plt.tick_params(axis='x', length=0)
    plt.ylabel('% Classification Accuracy', fontsize=20)
    plt.title('10-Fold Cross-Validation Incrase or Decrease in Cases - Sample ' + results['sample_name'], fontsize=20)
    plt.grid(axis='y', linewidth=0.4)
    plt.ylim([75, 101])
    offset = 0
    for mla in mlas:
        x = [offset for _ in range(len(results[mla]))]
        y = 100. * np.array(results[mla])
        if int(max(y)) <= 75:
            plt.scatter(offset+0, 76, color='green', marker='v')
        else:
            plt.scatter(x, y, color='green', marker='_', s=1000)
        plt.axvline(x=offset, color='grey', linewidth=0.3, alpha=0.3)
        offset += 4
    fig.text(0, -.20, 'Illustrates distribution of accuracies for each MLA.\nResults from 100 random runs.\nA down facing triangle indicates results below 75% accuracy'
             , fontsize=20)